In [1]:
import os
import json
import csv


In [2]:

# 定义目录和文件路径
dataset = 'PAF' # 'HARDVS', 'PAF', 'DVS128Gesture', 'SeAct'
root_dir = '/root/autodl-tmp/{}'.format(dataset)
base_dir = '/root/autodl-tmp/{}_Sampled_EZCLIP'.format(dataset)
# label_mapping_file = '/root/wj/EZ_CLIP/AFE/{}/{}_idx_to_label.json'.format(dataset) # just for SeACT
dataset_train_file = '/root/wj/EZ_CLIP/AFE/{}/{}_train.txt'.format(dataset, dataset)
dataset_val_file = '/root/wj/EZ_CLIP/AFE/{}/{}_val.txt'.format(dataset,dataset)
train_output_file = '/root/wj/EZ_CLIP/dataset_splits/{}/Zero-shot/train.txt'.format(dataset)
val_output_file = '/root/wj/EZ_CLIP/dataset_splits/{}/Zero-shot/val.txt'.format(dataset)
idx_mapping_file = '/root/wj/EZ_CLIP/AFE/{}/{}.json'.format(dataset,dataset)
output_csv_file_labels = '/root/wj/EZ_CLIP/lists/{}_labels.csv'.format(dataset)
# description_file = '/root/wj/EZ_CLIP/AFE/{}/{}_ds_cls.json'.format(dataset) # just for SeAct
output_csv_file_description = '/root/wj/EZ_CLIP/GPT_discription/{}_gpt_Class_discription_new.csv'.format(dataset)
output_file_paths = '/root/autodl-tmp/{}.txt'.format(dataset)


# SeAct.txt

In [ ]:

# 获取所有叶结点文件的绝对路径
def get_leaf_files(dir_path):
    leaf_files = []
    for root, dirs, files in os.walk(dir_path):
        for file in files:
            leaf_files.append(os.path.join(root, file))
    return leaf_files

# 将文件路径写入文本文件
def write_paths_to_file(file_paths, output_file):
    with open(output_file, 'w') as f:
        for path in file_paths:
            f.write(f"{path}\n")

# 获取所有叶结点文件的路径并写入文本文件
leaf_files = get_leaf_files(root_dir)
write_paths_to_file(leaf_files, output_file_paths)
print(f"文件路径已写入 {output_file_paths}")


# train.txt & val.txt

In [3]:
'''
# just for SeAct
# 读取label映射
with open(label_mapping_file, 'r') as f:
    label_mapping = json.load(f)
'''
with open(idx_mapping_file, 'r') as f:
    idx_mapping = json.load(f)


# 读取train和val文件
with open(dataset_train_file, 'r') as f:
    print(dataset_train_file)
    train_paths = ['/'.join((line.strip().split('/')[-2], line.strip().split('/')[-1].replace('.aedat',''))) for line in f.readlines()]

with open(dataset_val_file, 'r') as f:
    val_paths = ['/'.join((line.strip().split('/')[-2], line.strip().split('/')[-1].replace('.aedat',''))) for line in f.readlines()]

# 初始化输出内容
train_output = []
val_output = []

# 遍历目录并生成输出内容
for root, dirs, files in os.walk(base_dir):
    if root == base_dir:  # 只遍历根目录下的子目录
        for sub_dir in dirs: # arm crossing
            sub_dir_path = os.path.join(base_dir, sub_dir) # /root/autodl-tmp/{}_Sampled_EZCLIP/arm crossing
            for sub_sub_dir in os.listdir(sub_dir_path): # chenjieneng_1.1
                full_path = os.path.join(sub_dir_path, sub_sub_dir) # /root/autodl-tmp/{}_Sampled_EZCLIP/arm crossing/chenjieneng_1.1
                if os.path.isdir(full_path):
                    file_list = [f for f in os.listdir(full_path) if f.endswith('.jpg')]
                    num_files = len(file_list)
                    label_index = idx_mapping.get(sub_dir, -1)
                    output_line = f"{full_path} {num_files} {label_index}"
                    # print(output_line)
                    path_identifier = '/'.join(full_path.split('/')[-2:])  # 获取路径中的标识符
                    # print(path_identifier)
                    if path_identifier in train_paths:
                        train_output.append(output_line)
                    elif path_identifier in val_paths:
                        val_output.append(output_line)
# 写入train_output_file
with open(train_output_file, 'w') as f:
    for line in train_output:
        f.write(line + '\n')

# 写入val_output_file
with open(val_output_file, 'w') as f:
    for line in val_output:
        f.write(line + '\n')

print(f"train.txt and val.txt files have been created at {train_output_file} and {val_output_file}")


/root/wj/EZ_CLIP/AFE/PAF/PAF_train.txt
train.txt and val.txt files have been created at /root/wj/EZ_CLIP/dataset_splits/PAF/Zero-shot/train.txt and /root/wj/EZ_CLIP/dataset_splits/PAF/Zero-shot/val.txt


# labels.csv

In [4]:

# 生成label的CSV文件
with open(idx_mapping_file, 'r') as f:
    idx_mapping = json.load(f)

# 创建一个反向映射以便快速查找类名
idx_to_class_name = [[value,key] for key, value in idx_mapping.items()]


# 写入CSV文件
with open(output_csv_file_labels, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['id', 'name'])
    for row in idx_to_class_name:
        csvwriter.writerow(row)

print(f"CSV file has been created at {output_csv_file_labels}")


CSV file has been created at /root/wj/EZ_CLIP/lists/PAF_labels.csv


# description.txt

In [5]:
# 生成label的CSV文件
with open(idx_mapping_file, 'r') as f:
    idx_mapping = json.load(f)
ctx_prompt = 'A series of photos recording human action for '
# 创建一个反向映射以便快速查找类名
idx_classname_des = [[value,key,'\n\n' + ctx_prompt + key] for key, value in idx_mapping.items()]
# 写入CSV文件
with open(output_csv_file_description, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['SNo', 'Class Name', 'GPT3 discription'])
    for row in idx_classname_des:
        csvwriter.writerow(row)

print(f"CSV file has been created at {output_csv_file_description}")


CSV file has been created at /root/wj/EZ_CLIP/GPT_discription/PAF_gpt_Class_discription_new.csv
